# Inicialización y funciones

En primer lugar realizamos las importaciones que vamos ha necesitar.

In [1]:
import os

import keras
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from natsort import natsorted
from sklearn import model_selection
from tensorflow.contrib.keras import applications
from tensorflow.contrib.keras import layers, models

Using TensorFlow backend.


Definimos una variables globales que vamos a necesitar.

In [2]:
categories = 'faces/faces/train'
data_path = 'faces/faces'
n_classes = 151

Ordenamos las categorias.

In [3]:
names_categories = natsorted(os.listdir(categories))

Creamos una función que devuelve los datos y sus correspondientes etquetas para la fase de train.

In [4]:
def get_train_data(names_categories, data_path):
    X = []
    y = []
    # Se listan las categorias
    for i, c in enumerate(names_categories):
        # Sobre cada categoria se buscan lso archivos que pertenecesn
        for f in os.listdir(os.path.join(data_path, 'train', c)):
            img_path = os.path.join(data_path, 'train', c, f)
            img = image.load_img(img_path, target_size=(299, 299))
            x = image.img_to_array(img)
            # Se almacena la imagen como datos de entrada y al categoria como etiqueta
            X += [x]
            y += [i]

    y = np.array(y)

    return X, y

Creamos una función que devuelve los datos y sus correspondientes etquetas para la fase de test.

In [5]:
def get_test_data(names_categories, data_path):
    X_test = []
    y_test = []
    # se ordenan los datos de igual manera que las categorias
    for f in natsorted(os.listdir(os.path.join(data_path, 'test'))):
        # A partir del nombre de la imagen se toma la categoria y
        # se le asigna el mismo numero que en los datos de train
        img_path = os.path.join(data_path, 'test', f)
        name_category = f.split('.')[0]
        label = names_categories.index(name_category)
        # Se almacena la etiqueta en el vector de etiquetas
        y_test.append(label)
        # Se habre la imagen y se guarda en el vector de muestras
        img = image.load_img(img_path, target_size=(299, 299))
        x_test = image.img_to_array(img)

        X_test += [x_test]

    y_test = np.array(y_test)

    return X_test, y_test

# Ejecución

Carga de los datos.

In [6]:
X, y = get_train_data(names_categories, data_path)
X_test, y_test = get_test_data(names_categories, data_path)

Se preprocesan los datos para la que se encuentren acordes a la red ya preentrenada.

In [7]:
X = applications.inception_v3.preprocess_input(np.array(X))
X_test = applications.inception_v3.preprocess_input(np.array(X_test))

Se carga la red.

In [8]:
base_model = applications.inception_v3.InceptionV3(input_shape=(299, 299, 3), weights='imagenet', include_top=False)
# add a global spatial average pooling layer
x = layers.GlobalAveragePooling2D()(base_model.output)
# Add the prediction layer of size n_classes
predictions = layers.Dense(n_classes, activation='softmax')(x)
model = models.Model(inputs=base_model.input, outputs=predictions)

Se define que capas son entrenables.

In [9]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

Se realiza la separacion entre los datos de train y validación

In [10]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

Se define un optimizador y se realiza la compilacion del modelo.

In [11]:
opt = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Se define el aumentado de datos

In [12]:
# datagen_train = ImageDataGenerator(
#                 featurewise_center=True,
#                 featurewise_std_normalization=True,
#                 rotation_range=20,
#                 width_shift_range=0.2,
#                 height_shift_range=0.2,
#                 horizontal_flip=True
#                 )

Se entrena el modelo.

In [13]:
model.fit(x=x_train, y=y_train, batch_size=20, epochs=20, validation_data=(x_val, y_val), verbose=2)

Train on 2173 samples, validate on 544 samples
Epoch 1/20
67s - loss: 2.1125 - acc: 0.6199 - val_loss: 0.6949 - val_acc: 0.8327
Epoch 2/20
64s - loss: 0.1172 - acc: 0.9775 - val_loss: 0.1108 - val_acc: 0.9651
Epoch 3/20
67s - loss: 0.0261 - acc: 0.9949 - val_loss: 0.1550 - val_acc: 0.9540
Epoch 4/20
65s - loss: 0.0256 - acc: 0.9954 - val_loss: 0.6959 - val_acc: 0.8309
Epoch 5/20
66s - loss: 0.0162 - acc: 0.9977 - val_loss: 0.0429 - val_acc: 0.9835
Epoch 6/20
65s - loss: 0.0096 - acc: 0.9972 - val_loss: 0.2927 - val_acc: 0.9301
Epoch 7/20
64s - loss: 0.0032 - acc: 0.9995 - val_loss: 0.0510 - val_acc: 0.9816
Epoch 8/20
65s - loss: 0.0033 - acc: 0.9991 - val_loss: 0.4878 - val_acc: 0.8768
Epoch 9/20
64s - loss: 2.4912e-04 - acc: 1.0000 - val_loss: 0.0064 - val_acc: 0.9982
Epoch 10/20
63s - loss: 0.0102 - acc: 0.9972 - val_loss: 0.0125 - val_acc: 0.9982
Epoch 11/20
64s - loss: 0.0090 - acc: 0.9991 - val_loss: 0.1074 - val_acc: 0.9706
Epoch 12/20
64s - loss: 0.0017 - acc: 0.9991 - val_loss:

Se evaluan los datos de test.

In [14]:
print(model.evaluate(X_test, y_test))


299/299 [==============================] - 7s     
[0.048624822499236733, 0.98996655618067964]
